In [1]:
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os, sys, re, datetime
from pathlib import Path

pj_dir = Path(os.getcwd()).parent
data_dir = pj_dir/'data'
src_dir = pj_dir/'src'
sys.path.append(str(src_dir))

from matplotlib import pyplot as plt
plt.style.use("bmh")
import numpy as np
import pandas as pd
import dask.dataframe as dd
from scipy import stats 

from tqdm import tqdm

In [93]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.cluster import KMeans
import collections
from google.cloud import translate

In [99]:
creadential_path = Path.home()/'.gcp/ikedaosushi.json'
translate_client = translate.Client.from_service_account_json(creadential_path)

In [13]:
os.listdir(data_dir)

['tokyo-weather-2003-2012.csv',
 'frame.csv',
 'icecream.csv',
 'papercall.csv',
 'talk_list.csv']

In [10]:
df = pd.read_csv(data_dir/'talk_list.csv')

In [14]:
df_frame = pd.read_csv(data_dir/'frame.csv')

In [12]:
df.head()

,name,email,avatar,location,bio,talk_format,twitter,url,organization,shirt_size,...,abstract,description,notes,audience_level,tags,rating,state,confirmed,created_at,additional_info
0,Shagufta Gurmukhdas,methwanishagufta@gmail.com,https://secure.gravatar.com/avatar/0b5ca911f7c...,Unknown,I am a Tech-Speaker for Mozilla and have been ...,Talk (30 minutes),shaguftamethwan,NaN,HSBC Technology India,Women's L,...,"You only look once is a state-of-the-art, high...",YOLO is an algorithm for high speed image proc...,I am a Tech speaker for Mozilla and have given...,Intermediate,"[""python"", ""artificial intelligence"", ""deep le...",67.600000,declined,True,"June 05, 2018 08:27 UTC",キャンセル
1,Claudiu Popa,pcmanticore@gmail.com,https://secure.gravatar.com/avatar/a5561bef010...,Unknown,"Being a software engineer by craft, but future...",Talk (45 minutes),PCManticore,https://github.com/PCManticore,NaN,Men's M,...,"The clock is ticking for Python 2, we are gett...","Tick, tock, tick, tock! The clock is ticking f...",This talk is oriented around two major questio...,Intermediate,"[""Python 3"", ""Best practices""]",81.125000,declined,True,"June 14, 2018 13:05 UTC",キャンセル
2,Sanyam Khurana,sanyam.khurana01@gmail.com,https://secure.gravatar.com/avatar/b3b24db20c7...,Unknown,- CPython contributor and bug triage access on...,Talk (30 minutes),ErSanyamKhurana,http://www.SanyamKhurana.com/blog,NaN,Men's L,...,"In this talk, we'll see how we make a language...",You have got this super awesome REST API serve...,Pre-requisites:\n- Basic knowledge of Python/D...,All,"[""Django"", ""REST"", ""API"", ""localization""]",73.500000,declined,True,"May 18, 2018 12:55 UTC",キャンセル
3,Kajal Puri,kajalpuri95@gmail.com,https://secure.gravatar.com/avatar/9289b083916...,Unknown,Kajal Puri is working as a Data Scientist in F...,Talk (45 minutes),Agirlhasnofame,https://kajal-puri.github.io/,Fractal Analytics,Women's L,...,NLP is an exciting way to interpret the textua...,FastText has been open-sourced by Facebook in ...,"First of all, I'd request for a travel assista...",All,"[""open source"", ""facebook"", ""machinelearning"",...",70.285714,accepted,True,"May 24, 2018 07:14 UTC",NaN
4,Dror Ayalon,d.stamail@gmail.com,https://papercallio-production.s3.amazonaws.co...,"New-York, NY",[Dror Ayalon](https://www.drorayalon.com) is a...,Talk (30 minutes),drorayalon,https://www.drorayalon.com/,Google Creative Lab,Men's L,...,There is no better time than now to be a Pytho...,### Creative Music Applications in Python\nWe ...,Recent breakthroughs in neural networks resear...,All,"[""audio"", ""music"", ""machine learning"", ""dsp"", ...",80.333333,accepted,True,"June 03, 2018 16:16 UTC",NaN


In [62]:
df = df.query('state == "accepted"')

In [19]:
df.columns

Index(['name', 'email', 'avatar', 'location', 'bio', 'talk_format', 'twitter',
       'url', 'organization', 'shirt_size', 'title', 'abstract', 'description',
       'notes', 'audience_level', 'tags', 'rating', 'state', 'confirmed',
       'created_at', 'additional_info'],
      dtype='object')

In [66]:
variants_dict = {
    "artificialintelligence": "ai",
    "machinelearning": "ml",
    "python3": "python",
    "webprogrammingincludingframeworks(django": "django",
    "webアプリケーション開発": "web", 
    "serverlessapplicationmodel": "serverless"
}

In [67]:
corpus = []
for i, row in df.iterrows():
    tags = eval(row['tags'])
    for tag in tags:
        tag = tag.lower()
        tag = tag.replace(' ', '')
        tmp_tags = re.split("and|/", tag)
        for tmp_tag in tmp_tags:
            for k, v in variants_dict.items():
                tmp_tag = tmp_tag.replace(k, v)
            corpus.append(tmp_tag)

In [69]:
# collections.Counter(corpus)

In [70]:
sentence_corpuses = []
for i, row in df.iterrows():
    sentence_tags = []
    tags = eval(row['tags'])
    for tag in tags:
        tag = tag.lower()
        tag = tag.replace(' ', '')
        tmp_tags = re.split("and|/", tag)
        for tmp_tag in tmp_tags:
            for k, v in variants_dict.items():
                tmp_tag = tmp_tag.replace(k, v)
            sentence_tags.append(tmp_tag)
            
    s_corpus = collections.Counter(sentence_tags)
    sentence_corpuses.append(s_corpus)

In [75]:
v = DictVectorizer()
vec = v.fit_transform(sentence_corpuses).toarray()

In [77]:
vec.shape

(54, 130)

In [84]:
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(vec)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [85]:
kmeans.labels_

array([1, 3, 0, 0, 3, 0, 1, 0, 3, 0, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0, 0, 3,
       0, 0, 4, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 0, 1,
       1, 3, 1, 0, 1, 3, 0, 3, 0, 3], dtype=int32)

In [86]:
df['cluster'] = kmeans.labels_

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [88]:
df[['cluster', 'title', 'abstract']].to_csv(data_dir/'talklist_clusterd.csv')

### 上手く行かなかったので翻訳

In [104]:
df['abstract_en'] = df['abstract'].apply(lambda x: translate_client.translate(x,target_language='en')['translatedText'] )

In [156]:
df['title_en'] = df['title'].apply(lambda x: translate_client.translate(x,target_language='en')['translatedText'] )

In [167]:
df['description_en'] = df['description'].apply(lambda x: translate_client.translate(x,target_language='en')['translatedText'] )

In [150]:
1/54

0.018518518518518517

In [124]:
40/54

0.7407407407407407

In [138]:
df['tags_text'] = df['tags'].apply(lambda x: ' '.join(eval(x)))

In [168]:
s_texts = df['tags_text'] + ' ' + df['title_en'] + ' ' + df['abstract_en'] + '' + df['description_en']

In [199]:
cv = CountVectorizer(min_df=0.1, max_df=0.9, stop_words="english")
vector = cv.fit_transform(s_texts)

In [200]:
cv.get_feature_names()

['10',
 '39',
 'actually',
 'addition',
 'analysis',
 'application',
 'applications',
 'attention',
 'aws',
 'based',
 'basic',
 'basically',
 'best',
 'build',
 'building',
 'called',
 'case',
 'cases',
 'code',
 'common',
 'contents',
 'create',
 'data',
 'deep',
 'developed',
 'development',
 'difficult',
 'django',
 'does',
 'easy',
 'education',
 'environment',
 'example',
 'examples',
 'experience',
 'explain',
 'file',
 'flask',
 'following',
 'follows',
 'framework',
 'frameworks',
 'function',
 'functions',
 'future',
 'general',
 'good',
 'html',
 'https',
 'image',
 'implement',
 'implementation',
 'implemented',
 'including',
 'information',
 'introduce',
 'introduction',
 'kind',
 'know',
 'knowledge',
 'language',
 'large',
 'learn',
 'learning',
 'libraries',
 'library',
 'like',
 'll',
 'machine',
 'make',
 'making',
 'maths',
 'method',
 'model',
 'models',
 'multiple',
 'necessary',
 'need',
 'new',
 'open',
 'order',
 'packaging',
 'people',
 'possible',
 'practices'

In [186]:
kmeans = KMeans(n_clusters=6, random_state=0)
kmeans.fit(vector)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [187]:
df['cluster'] = kmeans.labels_

In [188]:
df[['cluster', 'title', 'abstract']].to_csv(data_dir/'talklist_clusterd.csv')

In [189]:
from sklearn.decomposition import LatentDirichletAllocation

In [196]:
import mglearn

In [201]:
lda =LatentDirichletAllocation(n_components=6,  max_iter=100, learning_method='batch', random_state=0, n_jobs=-1)
lda.fit(vector)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=100, mean_change_tol=0.001,
             n_components=6, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [202]:
lda.perplexity(vector)

106.03805961476195

In [203]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(cv.get_feature_names()) 

In [204]:
mglearn.tools.print_topics(topics=range(6), feature_names=feature_names, sorting=sorting, topics_per_chunk=6,n_words=30)


topic 0       topic 1       topic 2       topic 3       topic 4       topic 5       
--------      --------      --------      --------      --------      --------      
talk          data          quot          learning      quot          web           
programming   processing    aws           data          talk          development   
using         time          session       model         learning      use           
world         39            39            machine       web           application   
problem       django        development   deep          science       django        
language      using         technology    models        39            introduce     
library       code          using         talk          things        talk          
learning      function      learning      problem       using         39            
10            example       library       image         libraries     used          
39            functions     programming   science       including

In [208]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [212]:
[TaggedDocument(s.split(),tags = [i+200]) for i, s in enumerate(s_texts)]

[['open',
  'source',
  'facebook',
  'machinelearning',
  'python',
  'wordvector',
  'skipgram',
  'data',
  'science',
  'deeplearning',
  'fasttext',
  'naturallanguageprocessing',
  'Build',
  'text',
  'classification',
  'models',
  '(',
  'CBOW',
  'and',
  'Skip-gram)',
  'with',
  'FastText',
  'in',
  'python',
  'NLP',
  'is',
  'an',
  'exciting',
  'way',
  'to',
  'interpret',
  'the',
  'textual',
  'data',
  'especially',
  'when',
  'we',
  'know',
  'that',
  'computer',
  'neither',
  'speak',
  'nor',
  'understand',
  'any',
  'kind',
  'of',
  'human',
  'language.',
  'So,',
  'how',
  'do',
  'we',
  'represent',
  'each',
  'word',
  'of',
  'a',
  'language',
  'in',
  'such',
  'a',
  'unique',
  'numerical',
  'pattern',
  'and',
  'process',
  'it',
  'in',
  'quickest',
  'way',
  'possible.',
  'Answer',
  'is',
  'FastText',
  'library.FastText',
  'has',
  'been',
  'open-sourced',
  'by',
  'Facebook',
  'in',
  '2016',
  'and',
  'with',
  'its',
  '

In [214]:
data4d2v = [TaggedDocument(words = s.split(), tags = [i]) for i,s in enumerate(s_texts)]

In [230]:
m = Doc2Vec(documents= data4d2v, dm = 0, vector_size=100, window=8, min_count=3, workers=4)

In [231]:
vectors_list = [m.docvecs[n] for n in range(len(m.docvecs))]

In [232]:
kmeans = KMeans(n_clusters=6, random_state=0)
kmeans.fit(vectors_list)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [233]:
df['cluster'] = kmeans.labels_

In [234]:
df[['cluster', 'title', 'abstract']].to_csv(data_dir/'talklist_clusterd.csv')

# ここから割当
結局手動でクラスタリングした。。

In [235]:
os.listdir(data_dir)

['talklist_clusterd.csv',
 'tokyo-weather-2003-2012.csv',
 'frame.csv',
 '.DS_Store',
 'icecream.csv',
 'talklist.csv',
 'papercall.csv']

In [338]:
df = pd.read_csv(data_dir/'talklist.csv')

In [340]:
df = df[df['additional_info'].isnull()]

In [342]:
df['level'] = df['audience_level'].replace({"All": 0, "Beginner": 1, "Intermediate": 2, "Advanced": 3})

## 45分割当

In [345]:
df45 = df.query('talk_format == "Talk (45 minutes)"').copy()

### 枠から埋めていく

In [346]:
df45.sort_values(['level', 'rating'], ascending=[True, False], inplace=True)

In [347]:
df_assign45 = pd.DataFrame(np.zeros((4, 7)))
df_assign45.iloc[0:3, 2:4] = -1
df_assign45.iloc[3, 0] = -1

In [374]:
def describe(s):
    title = s['title']
    id_ = s['id']
    genre = int(s['genre'])
    level = int(s['level'])
    lang = s['lang']
    
    desc = "{}:G{}_L{}_{}_{}".format(id_, genre, level, lang, title)
    return desc

In [319]:
from collections import defaultdict

In [390]:
def assign(df, df_assign):

    assigned_ids = []
    row_genre_dic = defaultdict(list)
    row_lang_dic = defaultdict(list)
    n_row, n_col = df_assign.shape
    for c in range(n_col):
        for r in range(n_row):
            if df_assign.iloc[r, c] == -1:
                print('割当できない時間帯です')
                continue

            target_index = 0
            force_flg = False
            while True:
                df_rest = df.query('id not in @assigned_ids')
                if df_rest.shape[0] == 0:
                    print('割当てるトークがありません')
                    df_assign.iloc[r, c] = -1
                    break
                try:
                    target_s = df_rest.iloc[target_index]
                except IndexError:
                    print('割当できなかったので強制割当します')
                    target_index = 0
                    force_flg = True

                target_id = target_s['id']
                target_genre = target_s['genre']
                target_lang = target_s['lang']

                cond1 = target_genre in row_genre_dic[r]
                cond2 = row_lang_dic[r].count(target_lang) > int(n_col/2)

                if not force_flg and (cond1 or cond2) :
                    target_index += 1
                    continue

                assigned_ids.append(target_id)
                row_genre_dic[r].append(target_genre)
                row_lang_dic[r].append(target_lang)
                desc = describe(target_s)
                df_assign.iloc[r, c] = desc
                print(desc)
                break
    
    df_rest = df.query('id not in @assigned_ids')
    if df_rest.shape[0] > 0:
        print("割当てられていないトークがあります")
        for i, r in df.iterrows():
            print(describe(r))
        
        
    return df, df_assign

In [388]:
df45, df_assign45 = assign(df45, df_assign45)

29:G3_L0_en_Why you should care about types: Python Typing in the Facebook Backend
28:G3_L0_ja_オンザフライ高速化パッケージの比較：Numba, TensorFlow, Dask, etc
26:G3_L0_en_Migrating from Py2 application to Py3: first trial in MonotaRO / Python2 から Python3 への移植: MonotaRO での取り組み
割当できない時間帯です
25:G4_L0_ja_Sphinx-2.0 とドキュメントの未来
24:G0_L0_ja_niconicoにおけるコンテンツレコメンドの取り組み
22:G4_L0_ja_PyCon JP における子ども向けワークショップの活動事例と実施の意義
21:G1_L0_ja_Djangoアプリケーションにおけるトイル撲滅戦記
割当できない時間帯です
割当できない時間帯です
割当できない時間帯です
14:G2_L0_ja_「リモートペアプロでマントルを突き抜けろ！」AWS Cloud9でリモートペアプロ＆楽々サーバーレス開発
割当できない時間帯です
割当できない時間帯です
割当できない時間帯です
4:G0_L0_en_Build text classification models ( CBOW and Skip-gram) with FastText in python
13:G1_L0_ja_DjangoではじめるPyCharm実践入門
9:G4_L0_ja_Pythonを使ったハードウェア開発について
8:G0_L0_en_Probabilistic Programming and Bayesian Deep Learning (comparing Edward and ZhuSuan python libraries)
12:G4_L1_ja_あなたと私いますぐパッケージン
19:G2_L1_ja_Pythonistaに贈るコンテナ入門
27:G1_L1_ja_Webアプリケーションの仕組み
17:G1_L1_ja_Djangoだってカンバンつくれるもん(Django Channels + Vue)
16:G3_L2_ja_JVM上

In [369]:
df45.shape

(23, 15)

In [357]:
df_assign45

,0,1,2,3,4,5,6
0,G3_L0_en_Why you should care about types: Pyth...,G4_L0_ja_Sphinx-2.0 とドキュメントの未来,-1,-1,G1_L2_en_Integrate Full-text Search service wi...,G0_L2_ja_SymPyによる数式処理,G2_L1_ja_Pythonistaに贈るコンテナ入門
1,"G3_L0_ja_オンザフライ高速化パッケージの比較：Numba, TensorFlow, ...",G0_L0_en_Build text classification models ( CB...,-1,-1,G3_L2_ja_JVM上で動くPython3処理系cafebabepyの実装詳解,G4_L1_en_HomeSecurity with Python,G1_L1_ja_Webアプリケーションの仕組み
2,G3_L0_en_Migrating from Py2 application to Py3...,G0_L0_ja_niconicoにおけるコンテンツレコメンドの取り組み,-1,-1,G2_L2_en_Applying serverless architecture patt...,G4_L1_ja_あなたと私いますぐパッケージン,G4_L0_ja_Pythonを使ったハードウェア開発について
3,-1,G4_L0_ja_PyCon JP における子ども向けワークショップの活動事例と実施の意義,G0_L0_en_Probabilistic Programming and Bayesia...,G0_L3_ja_実践・競馬データサイエンス,G1_L2_ja_Django REST Framework におけるAPI実装プラクティス,G1_L1_ja_Djangoだってカンバンつくれるもん(Django Channels +...,G1_L0_ja_DjangoではじめるPyCharm実践入門


## 30分

In [353]:
df30 = df.query('talk_format == "Talk (30 minutes)"').copy()
df30.sort_values(['level', 'rating'], ascending=[True, False], inplace=True)

In [354]:
df_assign30 = pd.DataFrame(np.zeros((5, 7)))

In [391]:
df30, df_assign30 = assign(df30, df_assign30)

49:G4_L0_en_Fun with Python and Kanji
5:G0_L0_en_Creative Music Applications in Python
41:G4_L0_en_Make a Drone using RaspberryPi and Google VoiceKit by Python
57:G0_L0_ja_1次元畳み込みフィルターを利用した音楽データのオートエンコーダ
37:G0_L0_en_Visualizing Topic Models
52:G0_L0_en_Detecting offensive messages using Deep Learning: A micro-service based approach
36:G2_L0_ja_Jupyterで広がるPythonの可能性
56:G1_L0_en_Adding JWT Authentication to Python and Django REST Framework Using Auth0
34:G4_L0_ja_Pythonで解く大学入試数学
7:G1_L0_ja_Pythonで始めるウェブスクレイピング実践入門
30:G2_L0_en_Python, AWS and FinTech
31:G4_L0_ja_Python研修の作り方-Teaching Is Learning-
55:G0_L0_ja_メルカリにおける AI 活用事例
44:G3_L1_ja_Pythonでざっくり学ぶUnixプロセス
54:G4_L1_ja_AltJSとしてのPython - フロントエンドをPythonで書こう
6:G1_L1_ja_自分が欲しいものをPythonで書く方法(Python for Myself)
35:G1_L1_en_Notebook as Web API: Turn your notebook into Web API
42:G2_L2_ja_複数アプリケーションのプロセスとログを管理するための新しいツールと手法
50:G1_L2_en_From Data to Web Application: Anime Character Image Recognition with Transfer Learning
51:G2_L2_en_Django を Zap

In [392]:
df_assign30

,0,1,2,3,4,5,6
0,49:G4_L0_en_Fun with Python and Kanji,52:G0_L0_en_Detecting offensive messages using...,"30:G2_L0_en_Python, AWS and FinTech",6:G1_L1_ja_自分が欲しいものをPythonで書く方法(Python for Mys...,53:G3_L3_ja_C拡張と共に乗り切るPython 2→3移行術,39:G1_L2_en_The Modern OAuth 2.0,-1
1,5:G0_L0_en_Creative Music Applications in Python,36:G2_L0_ja_Jupyterで広がるPythonの可能性,31:G4_L0_ja_Python研修の作り方-Teaching Is Learning-,35:G1_L1_en_Notebook as Web API: Turn your not...,32:G4_L2_ja_Pythonによる異常検知入門,43:G0_L2_ja_Pythonistaの選球眼（せんきゅうがん） - エンジニアリング...,-1
2,41:G4_L0_en_Make a Drone using RaspberryPi and...,56:G1_L0_en_Adding JWT Authentication to Pytho...,55:G0_L0_ja_メルカリにおける AI 活用事例,42:G2_L2_ja_複数アプリケーションのプロセスとログを管理するための新しいツールと手法,33:G0_L2_ja_料理写真が美味しく撮れる！ 開発現場から覗くAI料理カメラの裏側,"45:G0_L1_en_Interpretable Machine Learning, ma...",-1
3,57:G0_L0_ja_1次元畳み込みフィルターを利用した音楽データのオートエンコーダ,34:G4_L0_ja_Pythonで解く大学入試数学,44:G3_L1_ja_Pythonでざっくり学ぶUnixプロセス,50:G1_L2_en_From Data to Web Application: Anim...,48:G0_L2_en_How to Data Wrangling? Tips for us...,40:G0_L1_ja_Pythonで「お絵描きパズル」を解いてみた。,-1
4,37:G0_L0_en_Visualizing Topic Models,7:G1_L0_ja_Pythonで始めるウェブスクレイピング実践入門,54:G4_L1_ja_AltJSとしてのPython - フロントエンドをPythonで書こう,51:G2_L2_en_Django を Zappaで構築してServerless Pyth...,38:G4_L2_ja_WILDCAT SDKは量子コンピュータビジネスの味方となるのか！？,47:G0_L1_en_Interactive Network Visualization ...,-1
